# Dashを使ったアプリケーションサンプル

In [ ]:
# Dashと必要なライブラリのインポート
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

# サンプルデータの作成
iris = px.data.iris()

# 主成分分析（PCA）で2次元に次元削減
pca = PCA(n_components=2)
X_pca = pca.fit_transform(iris.drop('species', axis=1))
iris['PCA1'] = X_pca[:, 0]
iris['PCA2'] = X_pca[:, 1]

app = dash.Dash(__name__)

# CSS Gridレイアウトを使ったビュー配置
app.layout = html.Div([
    
    html.Div([
        # 左: 次元削減ビュー
        html.Div([
            dcc.Graph(
                id='pca-scatter',
                figure=px.scatter(
                    iris, x='PCA1', y='PCA2', color='species',
                ),
                style={'height': '100%', 'width': '100%'}
            )
        ], style={
            'gridArea': 'left',
            'aspectRatio': '1/1',
            'minWidth': '350px',
            'minHeight': '350px',
            'padding': '8px',
        }),
        # 右: 上下分割
        html.Div([
            html.Div([
                dcc.Graph(
                    id='pcp',
                    style={'height': '100%', 'width': '100%'}
                )
            ], style={'height': '50%', 'paddingBottom': '8px'}),
            html.Div([
                dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in iris.columns],
                    page_size=5,
                    style_table={'height': '200px', 'overflowY': 'auto'},
                    style_cell={'textAlign': 'left', 'fontSize': 12}
                )
            ], style={'height': '50%'})
        ], style={
            'gridArea': 'right',
            'display': 'flex',
            'flexDirection': 'column',
            'minWidth': '350px',
            'padding': '8px',
        })
    ], style={
        'display': 'grid',
        'gridTemplateAreas': '"left right"',
        'gridTemplateColumns': '1fr 1fr',
        'gap': '0px',
        'height': '600px',
        'width': '100%'
    })
])

# 次元削減ビューで選択した点の主成分をPCPに反映、クラスを表に反映
def get_selected_indices(selectedData):
    if selectedData and 'points' in selectedData:
        return [p['pointIndex'] for p in selectedData['points']]
    return []

@app.callback(
    [Output('pcp', 'figure'), Output('table', 'data')],
    [Input('pca-scatter', 'selectedData')]
)
def update_views(selectedData):
    print("Selected Data:", selectedData)
    indices = get_selected_indices(selectedData)
    if indices:
        filtered = iris.iloc[indices]
    else:
        filtered = iris
    # PCP: 選択データのみ強調
    pcp_fig = px.parallel_coordinates(
        filtered,
        color=filtered['species'].astype('category').cat.codes,
        dimensions=['sepal_width', 'sepal_length', 'petal_width', 'petal_length'],
        color_continuous_scale=px.colors.diverging.Tealrose,
    )
    # 表: 選択データのみ表示
    table_data = filtered.to_dict('records')
    return pcp_fig, table_data

# Jupyter Notebook用のサーバー起動
app.run_server(jupyter_mode="tab")

Selected Data: None
